## Cross Entropy Loss and Adam Optimizer

What is the behavior of cross entropy loss and adam optimizer?

In [1]:
INPUT_TEXT = ['the quick brown fox jumped over the lazy dog']

### Calamancy

In [2]:
import calamancy

Calamancy = calamancy.load("tl_calamancy_md-0.1.0")

Calamancy

/home/nate/miniconda3/lib/python3.9/site-packages/spacy/util.py:918: UserWarning: [W094] Model 'tl_calamancy_md' (0.1.0) specifies an under-constrained spaCy version requirement: >=3.5.0. This can lead to compatibility problems with older versions, or as new spaCy versions are released, because the model may say it's compatible when it's not. Consider changing the "spacy_version" in your meta.json to a version range, with a lower and upper pin. For example: >=3.7.4,<3.8.0
  warnings.warn(warn_msg)
/home/nate/miniconda3/lib/python3.9/site-packages/spacy/util.py:918: UserWarning: [W094] Model 'tl_calamancy_lg' (0.1.0) specifies an under-constrained spaCy version requirement: >=3.5.0. This can lead to compatibility problems with older versions, or as new spaCy versions are released, because the model may say it's compatible when it's not. Consider changing the "spacy_version" in your meta.json to a version range, with a lower and upper pin. For example: >=3.7.4,<3.8.0
  warnings.warn(warn

In [3]:
import pandas as pd
import numpy as np
import tqdm
from torch import Tensor
from collections import Counter

def get_calamancy_tokens(data):
  # Allows it to work with both dataframes and
  # simple lists of strings
  if isinstance(data, pd.Series):
    data = data.values

  samples = []

  progress_bar = tqdm.tqdm(total=len(data))

  for sample in Calamancy.pipe(data):
    progress_bar.update(1)

    tokens = []
    for token in sample:
      tokens.append(token)

    samples.append(tokens)

  progress_bar.close()

  return samples

def get_token_vectors(tokens):
  vectors = []

  progress_bar = tqdm.tqdm(total=len(tokens))

  for sample in tokens:
    progress_bar.update(1)
    
    token_vectors = []
    # Check in case empty due to processing
    if not sample:
      token_vectors.append(np.zeros((200)))
    else:
      for token in sample:
        if token.has_vector:
          token_vectors.append(token.vector)
    token_vectors = Tensor(np.array(token_vectors))

    vectors.append(token_vectors)

  progress_bar.close()

  return vectors

def data_remove_stopwords(data):
  stopwords_list = open(
    './src/stopwords-tl.txt',
    'r',
  ).read().split('\n')
  stopwords_dict = Counter(stopwords_list)
  return [
    ' '.join([
      word for word in sample.split()
      if word not in stopwords_dict
    ])
    for sample 
    in data
  ]

/tmp/ipykernel_34760/1945608868.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [4]:
input_tokenized = get_calamancy_tokens(INPUT_TEXT)

100%|██████████| 1/1 [00:00<00:00, 24.34it/s]


In [5]:
input_tokenized

[[the, quick, brown, fox, jumped, over, the, lazy, dog]]

In [6]:
input_vectorized = get_token_vectors(input_tokenized)

100%|██████████| 1/1 [00:00<00:00, 88.41it/s]


In [7]:
input_vectorized

[tensor([[-0.0448,  0.2607,  0.1448,  ...,  0.4243, -0.1153, -0.0620],
         [ 0.5258, -0.3996,  0.0114,  ...,  0.4171, -0.3223,  0.1630],
         [-0.0630,  0.6724,  1.0455,  ...,  0.2032, -0.4797,  0.5314],
         ...,
         [-0.0448,  0.2607,  0.1448,  ...,  0.4243, -0.1153, -0.0620],
         [-0.1454,  0.5539,  0.2981,  ...,  1.6336, -0.2340, -0.1128],
         [ 0.1416, -0.5068,  0.7797,  ...,  0.2225, -0.7088, -0.3081]])]

In [8]:
input_vectorized[0].shape

torch.Size([9, 200])

### LSTM

In [9]:
import torch
from torch import optim 

INPUT_SIZE = 200
NUM_OF_HIDDEN_NODES = 50
OUTPUT_SIZE = 2

LEARNING_RATE = 0.01

OPTIMIZER = optim.Adam

DEVICE = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

In [10]:
from torch import nn

class LstmModel(nn.Module):
  def __init__(self):
    super().__init__()
    self.lstm = nn.LSTM(
      INPUT_SIZE,
      NUM_OF_HIDDEN_NODES,
      batch_first=True,
    )
    self.linear = nn.Linear(NUM_OF_HIDDEN_NODES, OUTPUT_SIZE)

    self.lstm_output = None
    self.lstm_hidden_state = None
    self.lstm_cell_state = None

  def forward(self, input):
    self.lstm_output, (self.lstm_hidden_state, self.lstm_cell_state) = self.lstm(input)

    linear_output = self.linear(self.lstm_output[:, -1])

    return linear_output

Lstm = LstmModel()

Lstm.to(DEVICE)

Lstm

LstmModel(
  (lstm): LSTM(200, 50, batch_first=True)
  (linear): Linear(in_features=50, out_features=2, bias=True)
)

### Loss

In [11]:
loss_function = nn.CrossEntropyLoss()

In [12]:
input_tensor = torch.stack(input_vectorized).to(DEVICE)

output = Lstm(input_tensor)

In [13]:
output

tensor([[0.1412, 0.2750]], device='cuda:0', grad_fn=<AddmmBackward0>)

In [14]:
from torch import LongTensor

output_loss = loss_function(
  output, 
  LongTensor([1]).to(DEVICE),
)

In [15]:
output_loss

tensor(0.6285, device='cuda:0', grad_fn=<NllLossBackward0>)

### Solve for Loss

Assume that actual value is 1

Hence, the actual probability distribution is [0, 1]

In [16]:
output_probabilities = torch.softmax(
  output,
  dim=1,
)

In [17]:
output_probabilities

tensor([[0.4666, 0.5334]], device='cuda:0', grad_fn=<SoftmaxBackward0>)

In [18]:
actual_probabilities = [0, 1]

predicted_probabilities = output_probabilities[0][0] * actual_probabilities[0] + output_probabilities[0][1] * actual_probabilities[1]

-np.log(predicted_probabilities.cpu().detach().numpy())

0.6285088

### Loss for Multiple Inputs

In [19]:
MULTIPLE_INPUTS = [
  'the quick brown fox jumped',
  'over the lazy dog near',
  'the bank of the river',
]

TARGET_INPUTS = [1, 0, 1]

In [20]:
multiple_input_tokens = get_calamancy_tokens(MULTIPLE_INPUTS)

100%|██████████| 3/3 [00:00<00:00, 78.22it/s]


In [21]:
multiple_input_tokens

[[the, quick, brown, fox, jumped],
 [over, the, lazy, dog, near],
 [the, bank, of, the, river]]

In [22]:
multiple_input_vectors = get_token_vectors(multiple_input_tokens)

100%|██████████| 3/3 [00:00<00:00, 2446.61it/s]


In [23]:
multiple_input_vectors

[tensor([[-4.4816e-02,  2.6072e-01,  1.4476e-01,  1.3985e-01,  8.2186e-01,
          -1.8453e-01,  3.9610e-01, -3.3248e-01, -1.8321e-03, -3.2702e-01,
          -1.4955e-01, -7.9797e-01,  3.2212e-01,  5.0457e-01, -1.4466e-02,
          -1.5588e-01, -5.8039e-01, -8.2334e-01, -1.2749e-01,  1.2681e-02,
           1.7029e-01,  2.0639e-01,  1.8592e-01,  9.1905e-02,  7.9670e-01,
           3.7350e-03, -1.1508e-01, -9.2960e-02, -5.0474e-02, -5.2370e-01,
           2.4938e-01,  5.0046e-02, -2.2510e-01,  8.2782e-01, -8.8028e-02,
          -3.1285e-01,  2.1367e-01, -1.2164e-01,  6.2164e-02,  3.9969e-02,
          -1.9242e-01,  5.9761e-01,  8.7907e-01,  2.6625e-01, -6.2292e-01,
           3.5723e-01, -5.6910e-01, -2.2975e-02, -4.4734e-01, -2.3245e-01,
          -1.6014e-02,  2.4307e-02, -2.9628e-01,  3.4902e-03,  2.8123e-01,
          -3.6569e-01, -5.6186e-02, -8.3109e-02,  3.7591e-01, -3.0810e-01,
          -3.9157e-01, -5.8018e-01,  4.4514e-02, -3.4728e-01, -1.4072e-02,
           9.6888e-02,  4

In [24]:
multiple_input_vectors[0].shape

torch.Size([5, 200])

In [25]:
multiple_input_tensor = torch.stack(multiple_input_vectors).to(DEVICE)

multiple_output = Lstm(multiple_input_tensor)

In [26]:
multiple_output

tensor([[0.1192, 0.1045],
        [0.0689, 0.1744],
        [0.1400, 0.0759]], device='cuda:0', grad_fn=<AddmmBackward0>)

In [27]:
multiple_loss = loss_function(
  multiple_output,
  LongTensor(TARGET_INPUTS).to(DEVICE)
)

In [28]:
multiple_loss

tensor(0.7245, device='cuda:0', grad_fn=<NllLossBackward0>)

Solving for Loss

In [29]:
multiple_softmax = torch.softmax(
  multiple_output,
  dim=1,
)

In [30]:
multiple_softmax

tensor([[0.5037, 0.4963],
        [0.4736, 0.5263],
        [0.5160, 0.4840]], device='cuda:0', grad_fn=<SoftmaxBackward0>)

In [31]:
negative_log = -np.log(
  multiple_softmax.cpu().detach().numpy(),
)

In [32]:
negative_log

array([[0.685826  , 0.70052224],
       [0.7472867 , 0.64178896],
       [0.6616183 , 0.7257025 ]], dtype=float32)

In [33]:
negative_log_sum = negative_log[0][1] + negative_log[1][0] + negative_log[2][1]

In [34]:
negative_log_sum

2.1735115

In [35]:
negative_log_sum / 3

0.7245038350423177

## loss.backward() and optimizer.step()

In [36]:
optimizer = OPTIMIZER(Lstm.parameters(), lr=LEARNING_RATE)

### Getting Parameter Values

In [37]:
print("-----")
for name, param in Lstm.named_parameters():
  if param.requires_grad: 
    print(name)
    print("Data:")
    print(param.data)
    print("Grad:")
    print(param.grad)
    print(f"Shape: {param.data.shape}")
    print("-----")

-----
lstm.weight_ih_l0
Data:
tensor([[ 0.0861,  0.0419, -0.1367,  ...,  0.0496,  0.0273,  0.1175],
        [-0.0919, -0.0815,  0.0015,  ...,  0.0817, -0.0987, -0.0244],
        [ 0.1151,  0.0516, -0.0128,  ..., -0.0300, -0.1070,  0.1008],
        ...,
        [-0.0939,  0.1310,  0.0965,  ...,  0.1209, -0.0100,  0.0700],
        [ 0.0940,  0.1238,  0.0672,  ..., -0.1023, -0.0976,  0.0371],
        [ 0.0433, -0.1408,  0.0970,  ..., -0.0617,  0.0175,  0.1103]],
       device='cuda:0')
Grad:
None
Shape: torch.Size([200, 200])
-----
lstm.weight_hh_l0
Data:
tensor([[ 0.0126,  0.0527, -0.1170,  ..., -0.0755, -0.1405, -0.0456],
        [-0.0288,  0.1148,  0.0838,  ...,  0.0740, -0.0059,  0.0544],
        [ 0.0362, -0.0175,  0.0392,  ..., -0.0416, -0.0988,  0.1178],
        ...,
        [ 0.1031, -0.0318, -0.0704,  ..., -0.0499,  0.0895,  0.0116],
        [ 0.0801,  0.1109, -0.0494,  ...,  0.0293, -0.1004, -0.0431],
        [ 0.0087, -0.0532,  0.1160,  ...,  0.0101,  0.0443, -0.0656]],
       

In [38]:
# Accumulate gradients in parameters
output_loss.backward()

In [39]:
# Note that each parameter has a grad value

print("---------")
for name, param in Lstm.named_parameters():
  if param.requires_grad: 
    print(name)
    print("Data:")
    print(param.data)
    print(f"Data Shape: {param.data.shape}")
    print("-----")
    print("Grad:")
    print(param.grad)
    print(f"Grad Shape: {param.grad.shape}")
    print("---------")

---------
lstm.weight_ih_l0
Data:
tensor([[ 0.0861,  0.0419, -0.1367,  ...,  0.0496,  0.0273,  0.1175],
        [-0.0919, -0.0815,  0.0015,  ...,  0.0817, -0.0987, -0.0244],
        [ 0.1151,  0.0516, -0.0128,  ..., -0.0300, -0.1070,  0.1008],
        ...,
        [-0.0939,  0.1310,  0.0965,  ...,  0.1209, -0.0100,  0.0700],
        [ 0.0940,  0.1238,  0.0672,  ..., -0.1023, -0.0976,  0.0371],
        [ 0.0433, -0.1408,  0.0970,  ..., -0.0617,  0.0175,  0.1103]],
       device='cuda:0')
Data Shape: torch.Size([200, 200])
-----
Grad:
tensor([[-1.0380e-03,  4.2402e-03,  1.0249e-03,  ...,  9.9626e-03,
         -7.3713e-04, -4.7700e-04],
        [-1.3368e-03,  3.9806e-03, -4.5642e-03,  ...,  6.7374e-04,
          4.1394e-03,  1.7101e-03],
        [ 1.4198e-05, -8.4338e-05,  1.3620e-04,  ...,  8.9446e-05,
         -1.2793e-04, -6.5994e-05],
        ...,
        [-1.3210e-04,  6.8898e-05,  2.1496e-05,  ...,  3.9388e-04,
         -3.8631e-05, -1.2061e-04],
        [ 3.8427e-05, -7.0212e-04,  

In [40]:
optimizer.step()

In [41]:
# Note that the parameter data has changed

print("---------")
for name, param in Lstm.named_parameters():
  if param.requires_grad: 
    print(name)
    print("Data:")
    print(param.data)
    print(f"Data Shape: {param.data.shape}")
    print("-----")
    print("Grad:")
    print(param.grad)
    print(f"Grad Shape: {param.grad.shape}")
    print("---------")

---------
lstm.weight_ih_l0
Data:
tensor([[ 9.6144e-02,  3.1924e-02, -1.4674e-01,  ...,  3.9571e-02,
          3.7325e-02,  1.2754e-01],
        [-8.1926e-02, -9.1543e-02,  1.1522e-02,  ...,  7.1677e-02,
         -1.0867e-01, -3.4445e-02],
        [ 1.0513e-01,  6.1566e-02, -2.2773e-02,  ..., -4.0040e-02,
         -9.6986e-02,  1.1082e-01],
        ...,
        [-8.3885e-02,  1.2098e-01,  8.6485e-02,  ...,  1.1091e-01,
         -2.3675e-05,  7.9966e-02],
        [ 8.4030e-02,  1.3378e-01,  5.7184e-02,  ..., -1.1233e-01,
         -8.7619e-02,  4.7074e-02],
        [ 5.3339e-02, -1.5082e-01,  1.0702e-01,  ..., -7.1652e-02,
          7.5266e-03,  1.0026e-01]], device='cuda:0')
Data Shape: torch.Size([200, 200])
-----
Grad:
tensor([[-1.0380e-03,  4.2402e-03,  1.0249e-03,  ...,  9.9626e-03,
         -7.3713e-04, -4.7700e-04],
        [-1.3368e-03,  3.9806e-03, -4.5642e-03,  ...,  6.7374e-04,
          4.1394e-03,  1.7101e-03],
        [ 1.4198e-05, -8.4338e-05,  1.3620e-04,  ...,  8.9446e-0

In [42]:
optimizer.zero_grad()

In [43]:
# Note that the gradients have been cleared

print("---------")
for name, param in Lstm.named_parameters():
  if param.requires_grad: 
    print(name)
    print("Data:")
    print(param.data)
    print(f"Data Shape: {param.data.shape}")
    print("-----")
    print("Grad:")
    print(param.grad)
    print("---------")

---------
lstm.weight_ih_l0
Data:
tensor([[ 9.6144e-02,  3.1924e-02, -1.4674e-01,  ...,  3.9571e-02,
          3.7325e-02,  1.2754e-01],
        [-8.1926e-02, -9.1543e-02,  1.1522e-02,  ...,  7.1677e-02,
         -1.0867e-01, -3.4445e-02],
        [ 1.0513e-01,  6.1566e-02, -2.2773e-02,  ..., -4.0040e-02,
         -9.6986e-02,  1.1082e-01],
        ...,
        [-8.3885e-02,  1.2098e-01,  8.6485e-02,  ...,  1.1091e-01,
         -2.3675e-05,  7.9966e-02],
        [ 8.4030e-02,  1.3378e-01,  5.7184e-02,  ..., -1.1233e-01,
         -8.7619e-02,  4.7074e-02],
        [ 5.3339e-02, -1.5082e-01,  1.0702e-01,  ..., -7.1652e-02,
          7.5266e-03,  1.0026e-01]], device='cuda:0')
Data Shape: torch.Size([200, 200])
-----
Grad:
None
---------
lstm.weight_hh_l0
Data:
tensor([[ 2.2553e-02,  4.2683e-02, -1.0702e-01,  ..., -6.5505e-02,
         -1.3047e-01, -5.5627e-02],
        [-3.8799e-02,  1.0480e-01,  9.3752e-02,  ...,  6.3976e-02,
          4.1062e-03,  4.4390e-02],
        [ 4.6242e-02, -7.

## Testing

In [47]:
for name, param in Lstm.named_parameters():
  print(name)

lstm.weight_ih_l0
lstm.weight_hh_l0
lstm.bias_ih_l0
lstm.bias_hh_l0
linear.weight
linear.bias
